In [ ]:
-- a
select  count(distinct countrycode)
from    empires
where   empire = 'Danish Empire';

In [ ]:
-- b
select  count(distinct CC1.countrycode)
from    countries_continents CC1
JOIN    countries_continents CC2 on CC1.countrycode = CC2.countrycode and CC1.continent <> CC2.continent and CC1.continent = 'Europe';

In [ ]:
-- c
select  cast(sum(CL.percentage*C.population/100) as integer)
from    countries_continents CC
join    countries_languages CL on CC.countrycode = CL.countrycode and CL.language = 'Spanish' and CC.continent = 'South America'
join    countries C on C.code = CC.countrycode and C.population > 1000000;

In [ ]:
-- d
select  count(*) from (
select  count(*)
from    empires E 
join    countries_languages CL on E.countrycode = CL.countrycode and E.empire = 'Benelux'
group by CL.language
having  count(distinct E.countrycode) = (select count(distinct countrycode) from empires where empire = 'Benelux') 
) temp;

## Homework 3

In [5]:
-- a control
select  count(*)
FROM    cities C
join    countries A on C.countrycode = A.code
where   C.population * 100 < A.population;

(1 row(s) affected)

Total execution time: 00:00:00.017

count
3264


In [4]:
-- a
select  count(*)
FROM    cities C
join    countries A on C.countrycode = A.code
where   C.population > (A.population * 0.5);

(1 row(s) affected)

Total execution time: 00:00:00.014

count
14


In [8]:
-- b control
select count (*) from (
select  count(*)
from    countries_languages
group by countrycode
having  sum(percentage) > 100 ) TMP;

(1 row(s) affected)

Total execution time: 00:00:00.007

count
2


In [55]:
-- b
select  count(*) from (
select  countrycode
from    countries_languages
group by countrycode
having  sum(percentage) < 100
union   
select  C.code
from    countries C
where   not exists (select * from countries_languages where countrycode = C.code) )tmp;

(1 row(s) affected)

Total execution time: 00:00:00.011

count
208


In [13]:
-- c control
select  *
from    extremecities 
where   countrycode = 'FRA';

(1 row(s) affected)

Total execution time: 00:00:00.007

countrycode,biggest,smallest
FRA,2125246,90674


In [46]:
drop view if exists extremecities;
create view extremecities as
SELECT  countrycode, max(population) as biggest, min(population) as smallest
from    cities 
group by countrycode;

-- c 
select  C.id
from    cities C
join    extremecities EC on C.countrycode = EC.countrycode
where   (C.population/EC.smallest) = ( 
    select  max(biggest/smallest)
    from    extremecities 
);

Commands completed successfully

Commands completed successfully

(1 row(s) affected)

Total execution time: 00:00:00.028

id
456


In [59]:
-- d
drop view if exists urbanpop cascade;
create view urbanpop as
select  countrycode, sum(population) as population
from    cities
group by countrycode
having  sum(population) > 1000000;

drop view if exists popratio;
create view popratio as
select  C.code as countrycode, 1.0*U.population / C.population as ratio
from    countries C 
join    urbanpop U on C.code = U.countrycode;

select  countrycode
from    popratio
where   ratio = (select max(ratio) from popratio);

NOTICE: drop cascades to view popratio

Commands completed successfully

Commands completed successfully

NOTICE: view "popratio" does not exist, skipping

Commands completed successfully

Commands completed successfully

(1 row(s) affected)

Total execution time: 00:00:00.038

countrycode
SGP
